In [1]:
import pandas as pd
import numpy as np
from scipy.special import factorial
import random

np.random.seed(42)

data = pd.read_csv('C:/Users/ROG/Desktop/component_month_counts.csv')
data['Month'] = pd.to_datetime(data['Month'])

# Define the full range of months for each component
all_months = pd.date_range(start=data['Month'].min(), end=data['Month'].max(), freq='MS')
expanded_data = pd.DataFrame([(comp, month) for comp in data['Component'].unique() for month in all_months],
                             columns=['Component', 'Month'])
expanded_data = expanded_data.merge(data, on=['Component', 'Month'], how='left').fillna(0)
expanded_data['Count'] = expanded_data['Count'].astype(int)

# Parameters
num_samples_adjusted = 5000
num_prediction_samples = 1000  # Number of prediction samples for the next month
lambda_alpha_hyperprior = 1.0  # Exponential rate for alpha hyperprior
lambda_beta_hyperprior = 1.0   # Exponential rate for beta hyperprior
decay_rate = 0.1               # Exponential decay rate (gamma)

# List to store results
predicted_results = []

# Perform adjusted MCMC and prediction for each component
components = expanded_data['Component'].unique()
for component in components:
    component_data = expanded_data[expanded_data['Component'] == component]
    component_counts = component_data['Count'].values
    
    # Calculate the time in months since each observation
    time_since_observation = (component_data['Month'].max() - component_data['Month']).dt.days // 30
    
    # Compute exponential decay weights
    weights = np.exp(-decay_rate * time_since_observation)
    
    alpha_samples = []
    beta_samples = []
    lambda_samples_adjusted = []
    
    # Initialize alpha, beta, and lambda with Exponential hyperprior for alpha and beta
    current_alpha = np.random.exponential(1 / lambda_alpha_hyperprior)
    current_beta = np.random.exponential(1 / lambda_beta_hyperprior)
    current_lambda = np.random.gamma(current_alpha, 1 / current_beta)

    for _ in range(num_samples_adjusted):
        # Propose new alpha and beta from Exponential distributions
        proposed_alpha = np.random.exponential(1 / lambda_alpha_hyperprior)
        proposed_beta = np.random.exponential(1 / lambda_beta_hyperprior)
        
        # Sample lambda from a Gamma distribution with proposed alpha and beta
        proposed_lambda = np.random.gamma(proposed_alpha, 1 / proposed_beta)

        # Compute weighted likelihood using exponential decay
        if proposed_alpha > 0 and proposed_beta > 0:
            current_likelihood = np.sum(weights * (component_counts * np.log(current_lambda) - current_lambda - np.log(factorial(component_counts))))
            proposed_likelihood = np.sum(weights * (component_counts * np.log(proposed_lambda) - proposed_lambda - np.log(factorial(component_counts))))
            
            # Hyperprior (Exponential) ratios
            prior_ratio_alpha = (lambda_alpha_hyperprior * np.exp(-lambda_alpha_hyperprior * proposed_alpha)) / (lambda_alpha_hyperprior * np.exp(-lambda_alpha_hyperprior * current_alpha))
            prior_ratio_beta = (lambda_beta_hyperprior * np.exp(-lambda_beta_hyperprior * proposed_beta)) / (lambda_beta_hyperprior * np.exp(-lambda_beta_hyperprior * current_beta))
            
            # Acceptance ratio
            acceptance_ratio = np.exp(proposed_likelihood - current_likelihood) * prior_ratio_alpha * prior_ratio_beta
            
            # Accept or reject the proposed alpha, beta, and lambda
            if np.random.rand() < acceptance_ratio:
                current_alpha = proposed_alpha
                current_beta = proposed_beta
                current_lambda = proposed_lambda

        # Store samples for analysis
        alpha_samples.append(current_alpha)
        beta_samples.append(current_beta)
        lambda_samples_adjusted.append(current_lambda)

    # Calculate mean alpha, beta, and lambda over samples
    mean_alpha = np.mean(alpha_samples)
    mean_beta = np.mean(beta_samples)
    mean_lambda = np.mean(lambda_samples_adjusted)
    
    # Predict next month's count using posterior samples of lambda
    predicted_counts = [np.random.poisson(lam) for lam in lambda_samples_adjusted[:num_prediction_samples]]
    
    # Calculate prediction summary statistics
    predicted_counts_mean = np.mean(predicted_counts)
    predicted_counts_std = np.std(predicted_counts)
    
    # Append results for this component
    predicted_results.append([component, mean_alpha, mean_beta, mean_lambda,
                              predicted_counts_mean, predicted_counts_std])

# Convert results to DataFrame and save to CSV
predicted_results_df = pd.DataFrame(predicted_results, columns=['Component', 'Mean_Alpha', 'Mean_Beta', 'Mean_Lambda',
                                                                'Predicted_Mean', 'Predicted_Std'])
predicted_results_df.to_csv('C:/Users/ROG/Desktop/output.csv', index=False)
print("CSV file 'predicted_counts_per_component_with_decay_and_hyperparameters.csv' has been created successfully.")


C:\Users\ROG\AppData\Local\Temp\ipykernel_55108\3477824549.py:63: RuntimeWarning: divide by zero encountered in log
  proposed_likelihood = np.sum(weights * (component_counts * np.log(proposed_lambda) - proposed_lambda - np.log(factorial(component_counts))))
C:\Users\ROG\AppData\Local\Temp\ipykernel_55108\3477824549.py:63: RuntimeWarning: invalid value encountered in multiply
  proposed_likelihood = np.sum(weights * (component_counts * np.log(proposed_lambda) - proposed_lambda - np.log(factorial(component_counts))))
C:\Users\ROG\AppData\Local\Temp\ipykernel_55108\3477824549.py:63: RuntimeWarning: divide by zero encountered in log
  proposed_likelihood = np.sum(weights * (component_counts * np.log(proposed_lambda) - proposed_lambda - np.log(factorial(component_counts))))
C:\Users\ROG\AppData\Local\Temp\ipykernel_55108\3477824549.py:63: RuntimeWarning: invalid value encountered in multiply
  proposed_likelihood = np.sum(weights * (component_counts * np.log(proposed_lambda) - proposed_lam

CSV file 'predicted_counts_per_component_with_decay_and_hyperparameters.csv' has been created successfully.


In [ ]:
import pandas as pd
import numpy as np
from scipy.special import factorial
import random

np.random.seed(42)
data = pd.read_csv('C:/Users/ROG/Desktop/component_month_counts.csv')
data['Month'] = pd.to_datetime(data['Month'])


all_months = pd.date_range(start=data['Month'].min(), end=data['Month'].max(), freq='MS')
expanded_data = pd.DataFrame([(comp, month) for comp in data['Component'].unique() for month in all_months],
                             columns=['Component', 'Month'])
expanded_data = expanded_data.merge(data, on=['Component', 'Month'], how='left').fillna(0)
expanded_data['Count'] = expanded_data['Count'].astype(int)

num_samples_adjusted = 5000
num_prediction_samples = 1000
lambda_alpha_hyperprior = 1.0 
lambda_beta_hyperprior = 1.0
decay_rate = 0.1         

predicted_results = []

components = expanded_data['Component'].unique()
for component in components:
    component_data = expanded_data[expanded_data['Component'] == component]
    component_counts = component_data['Count'].values

time_since_observation = (component_data['Month'].max() - component_data['Month']).dt.days // 30
weights = np.exp(-decay_rate * time_since_observation)

alpha_samples = []
beta_samples = []
lambda_samples_adjusted = []

current_alpha = np.random.exponential(1 / lambda_alpha_hyperprior)
current_beta = np.random.exponential(1 / lambda_beta_hyperprior)
current_lambda = np.random.gamma(current_alpha, 1 / current_beta)

for _ in range(num_samples_adjusted):
    proposed_alpha = np.random.exponential(1 / lambda_alpha_hyperprior)
    proposed_beta = np.random.exponential(1 / lambda_beta_hyperprior)
    proposed_lambda = np.random.gamma(proposed_alpha, 1 / proposed_beta)
    if proposed_alpha > 0 and proposed_beta > 0:
            
        current_likelihood = np.sum(weights * (component_counts * np.log(current_lambda) - current_lambda - np.log(factorial(component_counts))))
        proposed_likelihood = np.sum(weights * (component_counts * np.log(proposed_lambda) - proposed_lambda - np.log(factorial(component_counts))))
        
        # Hyperprior ratios for alpha and beta
        prior_ratio_alpha = (lambda_alpha_hyperprior * np.exp(-lambda_alpha_hyperprior * proposed_alpha)) / (lambda_alpha_hyperprior * np.exp(-lambda_alpha_hyperprior * current_alpha))
        prior_ratio_beta = (lambda_beta_hyperprior * np.exp(-lambda_beta_hyperprior * proposed_beta)) / (lambda_beta_hyperprior * np.exp(-lambda_beta_hyperprior * current_beta))
        
        # Acceptance ratio
        acceptance_ratio = np.exp(proposed_likelihood - current_likelihood) * prior_ratio_alpha * prior_ratio_beta
        
        # Accept or reject the proposal
        if np.random.rand() < acceptance_ratio:
            current_alpha = proposed_alpha
            current_beta = proposed_beta
            current_lambda = proposed_lambda

    alpha_samples.append(current_alpha)
    beta_samples.append(current_beta)
    lambda_samples_adjusted.append(current_lambda)

mean_alpha = np.mean(alpha_samples)
mean_beta = np.mean(beta_samples)
mean_lambda = np.mean(lambda_samples_adjusted)

predicted_counts = [np.random.poisson(lam) for lam in lambda_samples_adjusted[:num_prediction_samples]]
predicted_counts_mean = np.mean(predicted_counts)
predicted_counts_std = np.std(predicted_counts)

predicted_results.append([component, mean_alpha, mean_beta, mean_lambda, predicted_counts_mean, predicted_counts_std])

predicted_results_df = pd.DataFrame(predicted_results, columns=['Component', 'Mean_Alpha', 'Mean_Beta', 'Mean_Lambda', 'Predicted_Mean', 'Predicted_Std'])
predicted_results_df.to_csv('C:/Users/ROG/Desktop/output.csv', index=False)
print("CSV file has been created successfully.")


